In [1]:
# Import necessary libraries/packages
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Fix random seed
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [2]:
# Read data from csv file
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# le = LabelEncoder()
sc = StandardScaler()

In [18]:
# Preprocess data
x = data.drop('label', axis=1)
y = data['label'].values

In [5]:
def _predict(n, train_set, test_set, train_ans, test_ans, size, result:list, mode):
    knn = KNeighborsClassifier(n_neighbors=n, p=2)
    knn.fit(train_set, train_ans)
    train_pred = knn.predict(train_set)
    test_pred = knn.predict(test_set)
    
    current = {
        'n': n,
        'test size': size,
        'mode': mode,
        'Train': accuracy_score(train_ans, train_pred),
        'Test': accuracy_score(test_ans, test_pred)
        }
    result.append(current)

In [6]:

def _train(size, result:list):
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=size, random_state=1, stratify=y)
    sc.fit(x)
    x_train_std, x_test_std = sc.transform(x_train), sc.transform(x_test)
    
    print('train start')
    for i in range (2, 21):
        _predict(i, x_train, x_test, y_train, y_test, size, result, 'train')
    print('std train start')
    for i in range (2, 21):
        _predict(i, x_train_std, x_test_std, y_train, y_test, size, result, 'std train')


In [8]:
result = []
for i in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:
    _train(i, result)

train start
std train start
train start
std train start
train start
std train start
train start
std train start
train start
std train start
train start
std train start
train start
std train start
train start
std train start
train start
std train start
train start
std train start


In [21]:
pd.DataFrame(result).sort_values(by=['Train', 'Test'], ascending=False)

,n,test size,mode,Train,Test
1,3,0.50,train,0.993939,0.973333
209,2,0.75,std train,0.992718,0.937803
171,2,0.70,std train,0.991919,0.938528
39,3,0.55,train,0.991914,0.971366
285,2,0.85,std train,0.991903,0.903065
...,...,...,...,...,...
357,17,0.95,train,0.390244,0.310587
379,20,0.95,std train,0.365854,0.366709
358,18,0.95,train,0.365854,0.286352
359,19,0.95,train,0.341463,0.272321


In [30]:
# Get final model result

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.75, random_state=1, stratify=y)
sc.fit(x)
x_train_std, x_test_std = sc.transform(x_train), sc.transform(x_test)

knn = KNeighborsClassifier(n_neighbors=2, p=2)
# knn.fit(x_train, y_train)
knn.fit(x_train_std, y_train)
# res = knn.predict(test)
res = knn.predict(sc.transform(test))
res = pd.DataFrame(data=res, columns=['label'])
res.to_csv('submit.csv', index_label='id')